## Imports, Connect to Database

In [1]:
import datajoint as dj
import numpy as np
import json
from subprocess import call
import yaml
import sys, os
import pandas as pd
import ray

sys.path.append('./lib')
from utilities import *
sys.path.append('../lib')
from utils import run

In [2]:
dj.conn()

Connecting kui@ucsd-demo-db.datajoint.io:3306


DataJoint connection (connected) kui@ucsd-demo-db.datajoint.io:3306

In [3]:
schema = dj.schema('kui_diffusionmap')
schema.spawn_missing_classes()

In [7]:
stack = 'MD589'
yaml_file = 'shape_params-aws.yaml'
scripts_dir = os.environ['REPO_DIR']

## Create Process Table
--script  shapeolopy_code/scripts/Cell_generator.py

In [9]:
@schema
class Process(dj.Computed):
    definition="""
    -> Structure
    -----
    size_positive : int   #size of file
    size_negative : int   #size of file
    """
    bucket = "mousebrainatlas-data"
    credFiles = scripts_dir+'/Cell_datajoint/VaultBrain/credFiles_aws.yaml'
    client = get_s3_client(credFiles)
    def make(self, key):
        struc = (Structure & key).fetch1('structure')
        print('populating for ', struc, end='\n')
        for state in ['positive', 'negative']:
            item_name = state+'_s3_fp'
            key_item = 'size_'+state
            s3_fp = (Structure & key).fetch1(item_name)
            try:
                report = self.client.stat_object(self.bucket, s3_fp)
                key[key_item] = int(report.size/1000000)
            except:
                ! python3 $scripts_dir/Cell_generator.py $stack $struc $state $yaml_file
                upload_fp = 's3://'+self.bucket+'/'+s3_fp
                pkl_out_file = os.environ['ROOT_DIR']+'CSHL_cells_dm/'+stack+'/'+ struc + '/'+stack+'_'+struc+'_'+state+'.pkl'
                ! aws s3 cp $pkl_out_file $upload_fp
                print(pkl_out_file,upload_fp)
                report = self.client.stat_object(self.bucket, s3_fp)
                key[key_item] = int(report.size / 1000000)
        try:
            self.insert1(key)
        except:
            print('could not insert key=', key)


./lib/utilities.py:27: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  credFiles = yaml.load(open( credential_file_pointers,'r'))


In [8]:
Process.drop()

`kui_diffusionmap`.`__process` (4 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.


In [14]:
diffusion = Process()
diffusion.populate(reserve_jobs=True)


populating for  VCA
2019-05-24 13:37:09,445	WARNING worker.py:1341 -- WARNING: Not updating worker name since `setproctitle` is not installed. Install this with `pip install setproctitle` (or ray[debug]) to enable monitoring of worker processes.
2019-05-24 13:37:09,446	INFO node.py:497 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-24_13-37-09_445865_20912/logs.
2019-05-24 13:37:09,554	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:11288 to respond...
2019-05-24 13:37:09,664	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:46317 to respond...
2019-05-24 13:37:09,665	INFO services.py:806 -- Starting Redis shard with 30.0 GB max memory.
2019-05-24 13:37:09,677	INFO node.py:511 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-05-24_13-37-09_445865_20912/logs.
2019-05-24 13:37:09,677	WARNING services.py:1318 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 36878774272 

NoSuchKey: NoSuchKey: message: The specified key does not exist.

In [11]:
diffusion.fetch(as_dict=True)

[OrderedDict([('id', 1), ('size_positive', 103), ('size_negative', 1615)]),
 OrderedDict([('id', 4), ('size_positive', 1073), ('size_negative', 6969)]),
 OrderedDict([('id', 13), ('size_positive', 4053), ('size_negative', 19081)]),
 OrderedDict([('id', 14), ('size_positive', 85), ('size_negative', 3780)])]